**Question 1** Write a memoizing function and test it 

In [72]:
# Creare a memoizing function

def memoize(fnx):
    cache = {}
    kw_start_mark = (object(),)
    def inner(*args, **kwargs):
        key = args
        if kwargs:
            key += kw_start_mark
            for kw_item in sorted(kwargs.items()):
                # kw start mark let's us make the cache key flat (i.e. k1, v1, k2, v2) instead of
                # nested ((k1, v1), (k2, v2)), an optimization taken from the actual python library
                key += tuple(kw_item,)
        if key in cache:
            result = cache[key]
        else:
            result = fnx(*args, **kwargs)
            cache[key] = result
        return result
    return inner



In [73]:
def fibb(s):
    if s in (0, 1):
        return 1
    else:
        return fibb(s - 1) + fibb(s - 2)

@memoize
def m_fibb(s):
    if s in (0, 1):
        return 1
    else:
        return m_fibb(s - 1) + m_fibb(s - 2)


In [74]:
%time fibb(32)

CPU times: user 1.22 s, sys: 9.52 ms, total: 1.23 s
Wall time: 1.24 s


3524578

In [75]:
%time m_fibb(32)

CPU times: user 64 µs, sys: 0 ns, total: 64 µs
Wall time: 67 µs


3524578

**Question 2:** Try to memoize a function from your standard library that you normally use to produce random numbers. Does it work?

In [77]:
import random
m_randrange =  memoize(random.randrange)
%time result_1 = m_randrange(100)
%time result_2 = m_randrange(100)
%time result_3 = m_randrange(100)
# Should not be equal but they are...uh oh
result_1 == result_2 == result_3

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 17.2 µs
CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10 µs
CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 11.2 µs


True

**Question 3:** Most random number generators can be initialized with a seed. Implement a function that takes a seed, calls the random number generator with that seed, and returns the result. Memoize that function. Does it work?

In [79]:
def random_with_seed(seed, end):
    random.seed(seed)
    result = random.randrange(end)
    random.seed(None)
    return result

In [86]:
%time result_1 = random_with_seed(10, 100)
%time result_2 = random_with_seed(10, 100)
%time result_3 = random_with_seed(10, 100)
%time result_4 = random_with_seed(12, 100)
# 1-3 should equal, but not 4
result_1 == result_2 == result_3 != result_4

CPU times: user 435 µs, sys: 706 µs, total: 1.14 ms
Wall time: 484 µs
CPU times: user 338 µs, sys: 660 µs, total: 998 µs
Wall time: 413 µs
CPU times: user 314 µs, sys: 360 µs, total: 674 µs
Wall time: 389 µs
CPU times: user 348 µs, sys: 326 µs, total: 674 µs
Wall time: 496 µs


True

In [87]:
m_random_with_seed = memoize(random_with_seed)

%time result_1 = m_random_with_seed(10, 100)
%time result_2 = m_random_with_seed(10, 100)
%time result_3 = m_random_with_seed(10, 100)
%time result_4 = m_random_with_seed(12, 100)
# 1-3 should equal, but not 4
result_1 == result_2 == result_3 != result_4

CPU times: user 251 µs, sys: 549 µs, total: 800 µs
Wall time: 348 µs
CPU times: user 8 µs, sys: 1e+03 ns, total: 9 µs
Wall time: 11.9 µs
CPU times: user 9 µs, sys: 1 µs, total: 10 µs
Wall time: 11.9 µs
CPU times: user 539 µs, sys: 611 µs, total: 1.15 ms
Wall time: 603 µs


True

**Question 4:** Which of these C++ functions are pure? Try to memoize them and observe what happens when you call them multiple times: memoized and not.

Skipping implementation. Not worth the time.

#### 1) The fibb function from the example in the text.
Yes, pure.

#### 2)
```c
std::getchar()

```
No, not pure. Memoized will produce different output for same input, which is Void
#### 3)
```c
bool f() { 
    std::cout << "Hello!" << std::endl;
    return true; 
}
```
Not pure. Has side effect. Memoized will always return true though.

#### 4) 

```c
int f(int x)
{
    static int y = 0;
    y += x;
    return y;
}
```

Not pure: Function output changes every time due to static. Memoized would return the same result for the same input, but things would get weird fast since f(1), f(2), f(1) would produce 3 distinct results where as mem_f(1), mem_f(2), mem_f(1) would produce the same result for input = 1